In [7]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.preprocessing as pp
import utils.io_model as im
import utils.join_race_data as join_race_data
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path
import tensorflow as tf

In [2]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

True

In [3]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
TARGET_DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/test_data'
TRAIN_DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/train_data'

# 前処理

In [4]:
df_target_horse = rd.read_target_horse_csv(TARGET_DATA_PATH)
df_target_race  = rd.read_target_race_csv(TARGET_DATA_PATH)
df_target = pd.merge(df_target_horse, df_target_race, on='race_id', how='left')

In [21]:
# ターゲットデータに過去3レース分の情報を追加
df_past_data = rd.read_horse_race_csv(TRAIN_DATA_PATH)
columns_past_data = [c for c in df_past_data.columns if "-" not in c]
df_for_prediction = join_race_data.join_n_race_for_test_data(df_past_data[columns_past_data], df_target, 3)

horse_race_20201128.csv


c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (17,65,113,157,161,205,209,253,257) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [22]:
df_for_prediction.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,...,hukusyo_second-3,hukusyo_third-3,wakuren-3,umaren-3,wide_1_2-3,wide_1_3-3,wide_2_3-3,umatan-3,renhuku3-3,rentan3-3
0,2.020061e+11,1,5,9,2016104750,牝4,55.0,1102,2:35.0,NaN,...,110,130,340,400,180,230,210,780,400,"1,770"
1,2.020061e+11,2,7,14,2015104793,牝5,55.0,1126,2:35.0,クビ,...,280,200,"1,260","2,290",930,490,"1,460","3,610","4,260","21,050"
2,2.020061e+11,3,7,13,2015105075,牡5,57.0,5339,2:35.1,クビ,...,320,630,"1,480","1,780",710,"1,660","4,260","2,470","16,410","49,110"
3,2.020061e+11,4,4,7,2015105046,牝5,55.0,1014,2:35.5,2.1/2,...,160,140,"1,130","1,110",370,340,390,"1,970","1,350","7,810"
4,2.020061e+11,5,3,5,2016104854,牡4,57.0,666,2:35.6,1/2,...,390,120,610,"4,680","1,460",320,720,"7,640","3,070","23,510"


## 前処理

In [ ]:
columns = ["race_course", "weather", "ground_status", 
           "where_racecourse", "race_class", "running_condition", 
           "frame_number", "horse_number",
           "sex_and_age", "burden_weight", "rider_id", 
           "tamer_id", "horse_weight",
           "total_horse_number_x", 
           "rank-1", "rank-2", "rank-3",
           "total_horse_number_x-1", "total_horse_number_x-2","total_horse_number_x-3",
           "goal_time-1", "goal_time-2", "goal_time-3",
           "last_time-1", "last_time-2", "last_time-3", 
           "half_way_rank-1", "half_way_rank-2", "half_way_rank-3", 
           "prize-1", "prize-2", "prize-3"]

df = df[columns]

In [ ]:
df["where_racecourse"] = df["where_racecourse"].map(pp.extract_place)

df["sex"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
df["age"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[1:])

df["goal_time-1"] = df["goal_time-1"].map(pp.to_seconds)
df["goal_time-2"] = df["goal_time-2"].map(pp.to_seconds)
df["goal_time-3"] = df["goal_time-3"].map(pp.to_seconds)

df["horse_weight"] = df["horse_weight"].map(pp.extract_weight).astype(np.int64)

df["prize-1"] = df["prize-1"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize-2"] = df["prize-2"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize-3"] = df["prize-3"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)

df["kyakusitu-1"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-1"].values, df["half_way_rank-1"])]
df["kyakusitu-2"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-2"].values, df["half_way_rank-2"])]
df["kyakusitu-3"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-3"].values, df["half_way_rank-3"])]

# 欠損値処理
df = df.replace('---', -1)
df = df.fillna(-1)

In [ ]:
df["rank-1"] = pp.make_label(df["rank-1"].values, df["total_horse_number_x-1"].values)
df["rank-2"] = pp.make_label(df["rank-2"].values, df["total_horse_number_x-2"].values)
df["rank-3"] = pp.make_label(df["rank-3"].values, df["total_horse_number_x-3"].values)

In [ ]:
def fill_missing_columns(df_a, df_b):
    columns_for_b = set(df_a.columns) - set(df_b.columns)
    for column in columns_for_b:
        df_b[column] = 0

    columns_for_a = set(df_b.columns) - set(df_a.columns)
    for column in columns_for_a:
        df_a[column] = 0

In [ ]:
# targetデータを予測に用いる為に、targetデータに存在していない学習時に用いたカテゴリデータを追加
df_used_learning = pd.read_csv(DATA_PATH+"/learning/horse_race.csv")

columns = ["race_course", "weather", "ground_status", 
           "where_racecourse", "race_class", "running_condition", 
           "frame_number", "horse_number",
           "sex_and_age", "burden_weight", "rider_id", 
           "tamer_id", "horse_weight",
           "total_horse_number_x", 
           "rank-1", "rank-2", "rank-3",
           "total_horse_number_x-1", "total_horse_number_x-2","total_horse_number_x-3",
           "goal_time-1", "goal_time-2", "goal_time-3",
           "last_time-1", "last_time-2", "last_time-3", 
           "half_way_rank-1", "half_way_rank-2", "half_way_rank-3", 
           "prize-1", "prize-2", "prize-3"]

df_used_learning = df_used_learning[columns]

df_used_learning["where_racecourse"] = df_used_learning["where_racecourse"].map(pp.extract_place)

df_used_learning["sex"] = df_used_learning["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
df_used_learning["age"] = df_used_learning["sex_and_age"].map(lambda sex_and_age: sex_and_age[1:])

df_used_learning["goal_time-1"] = df_used_learning["goal_time-1"].map(pp.to_seconds)
df_used_learning["goal_time-2"] = df_used_learning["goal_time-2"].map(pp.to_seconds)
df_used_learning["goal_time-3"] = df_used_learning["goal_time-3"].map(pp.to_seconds)

df_used_learning["horse_weight"] = df_used_learning["horse_weight"].map(pp.extract_weight).astype(np.int64)

df_used_learning["prize-1"] = df_used_learning["prize-1"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df_used_learning["prize-2"] = df_used_learning["prize-2"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df_used_learning["prize-3"] = df_used_learning["prize-3"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)

df_used_learning["kyakusitu-1"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df_used_learning["total_horse_number_x-1"].values, df_used_learning["half_way_rank-1"])]
df_used_learning["kyakusitu-2"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df_used_learning["total_horse_number_x-2"].values, df_used_learning["half_way_rank-2"])]
df_used_learning["kyakusitu-3"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df_used_learning["total_horse_number_x-3"].values, df_used_learning["half_way_rank-3"])]

# 欠損値処理
df_used_learning = df_used_learning.replace('---', -1)
df_used_learning = df_used_learning.fillna(-1)

df_used_learning["rank-1"] = pp.make_label(df_used_learning["rank-1"].values, df_used_learning["total_horse_number_x-1"].values)
df_used_learning["rank-2"] = pp.make_label(df_used_learning["rank-2"].values, df_used_learning["total_horse_number_x-2"].values)
df_used_learning["rank-3"] = pp.make_label(df_used_learning["rank-3"].values, df_used_learning["total_horse_number_x-3"].values)

input_columns = ["race_course", "weather", "ground_status", 
                 "where_racecourse", "race_class", "running_condition", 
                 "frame_number", "horse_number",
                 "sex", "age", "burden_weight", "rider_id", 
                 "tamer_id", "horse_weight",
                 "rank-1", "rank-2", "rank-3", 
                 "goal_time-1", "goal_time-2", "goal_time-3",
                 "last_time-1", "last_time-2", "last_time-3", 
                 "kyakusitu-1", "kyakusitu-2", "kyakusitu-3", 
                 "prize-1", "prize-2", "prize-3"]

df_used_learning = pp.one_hot_encoding(df_used_learning[input_columns])
df = pp.one_hot_encoding(df[input_columns])

fill_missing_columns(df, df_used_learning)

# 予測

In [ ]:
model = im.read_model("second_model")

In [ ]:
x = np.array(df)

In [ ]:
test = []
for x_input, umaban, horse_id in zip(x, list(range(len(df_target["horse_id"]))), list(df_target["horse_id"])):
    pred = model.predict(x_input.reshape(1, -1))
    if umaban == 1 or umaban == 2 or umaban == 4 or umaban == 5:
        test.append(x_input)
    prediction_class = np.argmax([pred[0][0], pred[0][1], pred[0][2]])
    print("class: %d, (%f, %f, %f), %d (%d)" % (prediction_class, pred[0][0], pred[0][1], pred[0][2], umaban + 1, horse_id))